Base colab code source for LDA model taken from: https://colab.research.google.com/github/MIND-Lab/OCTIS/blob/master/examples/OCTIS_LDA_training_only.ipynb

BERTopic evaluation package: https://github.com/MaartenGr/BERTopic_evaluation

In [ ]:
!pip install octis
!pip install .[bertopic]
!pip install bertopic==v0.9.4
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 7.4 MB/s 
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 16.5 MB 55.9 MB/s 
     |████████████████████████████████| 24.1 MB 57 kB/s 
     |████████████████████████████████| 170 kB 42.3 MB/s 
     |████████████████████████████████| 24.9 MB 30.9 MB/s 
     |████████████████████████████████| 100 kB 5.2 MB/s 
     |████████████████████████████████| 5.8 MB 44.1 MB/s 
     |████████████████████████████████| 1.3 MB 49.7 MB/s 
     |████████████████████████████████| 182 kB 52.3 MB/s 
     |████████████████████████████████| 7.6 MB 38.0 MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp38-cp38-linux_x86_64.whl size=233370 sha256=72f823f613b8593426af101cfc6b2abf10b1ad046517cd6d641dcd025d4dd12e
  Stored in directory: /root/.cache/pip/wheels/41/00/e7/b328c0f5e4bbb9ac8ba5a72e56b1749be63dc1ab1c9321

In [ ]:
from octis.models.LDA import LDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from sklearn.feature_extraction.text import CountVectorizer
# from evaluation import Trainer
# from data import DataLoader

In [ ]:
# Define dataset
dataset = Dataset()
dataset.fetch_dataset("20NewsGroup")

print(dataset.get_vocabulary())

NUM_TOPICS = 20

['ability', 'absolute', 'absolutely', 'abuse', 'accept', 'acceptable', 'access', 'accident', 'accomplish', 'accord', 'account', 'accurate', 'achieve', 'act', 'action', 'active', 'activity', 'actual', 'add', 'addition', 'additional', 'address', 'adjust', 'administration', 'admit', 'adult', 'advance', 'advanced', 'advantage', 'advice', 'advocate', 'affect', 'afford', 'afraid', 'age', 'agency', 'agent', 'ago', 'agree', 'agreement', 'ahead', 'aid', 'aim', 'air', 'algorithm', 'alive', 'alt', 'alternative', 'amazing', 'amendment', 'american', 'amount', 'analysis', 'ancient', 'angel', 'animal', 'announce', 'announcement', 'annual', 'answer', 'anti', 'anymore', 'app', 'apparent', 'apparently', 'apple', 'application', 'apply', 'approach', 'approve', 'arab', 'archive', 'area', 'aren', 'argue', 'argument', 'arise', 'arm', 'armed', 'armenian', 'army', 'arrest', 'arrive', 'art', 'article', 'aspect', 'assault', 'assert', 'assist', 'associate', 'assume', 'assumption', 'assure', 'atheism', 'atheist', 

In [ ]:
# Create Model
model = LDA(num_topics=NUM_TOPICS, alpha=0.1)

In [ ]:
# Train the model using default partitioning choice 
output = model.train_model(dataset)

print(*list(output.keys()), sep="\n") # Print the output identifiers

topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix


In [ ]:
for t in output['topics'][:5]:
  print(" ".join(t))

game team win year play good player make season time
car buy money problem company pay year work sell good
chip card mode bit run time communication bank work system
make thing good time point problem people find light back
book jewish patient arab disease page information title greek program


In [ ]:
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_npmi')
umass = Coherence(texts=dataset.get_corpus(), topk=10, measure='u_mass')
cv = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_v')
uci = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_uci')

In [ ]:
# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [ ]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(output)
print("Coherence: "+str(npmi_score))
umass_score = umass.score(output)
print("Coherence: "+str(umass_score))
cv_score = cv.score(output)
print("Coherence: "+str(cv_score))
uci_score = uci.score(output)
print("Coherence: "+str(uci_score))

Topic diversity: 0.74
Coherence: 0.04804667849438671
Coherence: -2.2445109826639307
Coherence: 0.519133103675679
Coherence: 0.017321758979920034


In [ ]:
%%capture
from sentence_transformers import SentenceTransformer

# Prepare data
dataset, custom = "20NewsGroup", False
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(data, show_progress_bar=True)

In [ ]:
import pickle

with open('embeddings.pickle', 'wb') as handle:
    pickle.dump((dataset, custom, embeddings), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('embeddings.pickle', 'rb') as handle:
    dataset, custom, embeddings = pickle.load(handle)

In [ ]:
from evaluation import Trainer
print(str(dataset))
params = {
    "embedding_model": "all-mpnet-base-v2",
    "nr_topics": [NUM_TOPICS for i in range(5)],
    "min_topic_size": 15,
    "diversity": None,
    "verbose": True
}

trainer = Trainer(dataset=dataset,
                  model_name="BERTopic",
                  params=params,
                  bt_embeddings=embeddings,
                  custom_dataset=custom,)
                  # verbose=True)
results = trainer.train(save=f"BERTopic_news_{1}")

20NewsGroup


2022-12-05 00:41:07,738 - BERTopic - Reduced dimensionality with UMAP
2022-12-05 00:41:08,725 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-12-05 00:41:27,239 - BERTopic - Reduced number of topics from 85 to 21


Results123
npmi: 0.18182166234282388
umass: -1.7965300794488983
cv: 0.7425894106367811
uci: 1.4153266737303767
diversity: 0.9
 


2022-12-05 00:42:25,919 - BERTopic - Reduced dimensionality with UMAP
2022-12-05 00:42:26,854 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-12-05 00:42:46,870 - BERTopic - Reduced number of topics from 86 to 21


Results123
npmi: 0.18374540244260112
umass: -1.8825536021191254
cv: 0.7416305115371481
uci: 1.4124947917432362
diversity: 0.88
 


...

Results123
npmi: 0.1855918150331591
umass: -1.8944230857100421
cv: 0.7527964370622736
uci: 1.44410616962557
diversity: 0.905
 


2022-12-05 00:46:12,068 - BERTopic - Reduced dimensionality with UMAP
2022-12-05 00:46:12,980 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-12-05 00:46:32,984 - BERTopic - Reduced number of topics from 89 to 21


Results123
npmi: 0.17778786299342791
umass: -1.8798098878148835
cv: 0.7342561860382728
uci: 1.3821971649500149
diversity: 0.92
 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>